<h1>Research Data</h1>
<h2>Research Question</h2>
<p>Does the content of a tweet serve as a predictor for what 'description' it will fall under? The description's are related to particular job_ids and the job_ids contain many tweets.</p>
<h2>The Process</h2>

In [1]:
import pymysql as mdb
import pandas as pd

connection = mdb.connect('128.206.116.195', 'tg4_ro', '?3stEt7!3hUbRa-R', 'tw4_db')

In [2]:
jobs = [
   {
      "job_id":"1160",
      "query":"q=%23tornado",
      "description":"Weather"
   },
   {
      "job_id":"2335",
      "query":"q=%23earthquake",
      "description":"Weather"
   },  
   {
      "job_id":"2616",
      "query":"q=autism",
      "description":"Autism"
   },
   {
      "job_id":"4219",
      "query":"q=%23teaparty",
      "description":"Election"
   },
   {
      "job_id":"4220",
      "query":"q=%23election",
      "description":"Election"
   },
   {
      "job_id":"4221",
      "query":"q=%23voterfraud",
      "description":"Election"
   },   
   {
      "job_id":"4253",
      "query":"q=%23mizzouunited",
      "description":"Mizzou"
   },
   {
      "job_id":"4257",
      "query":"q=%23MIZZOU",
      "description":"Mizzou"
   },
   {
      "job_id":"4258",
      "query":"q=%23prayformizzou",
      "description":"Mizzou"
   },
]

In [3]:
for job in jobs:
    sql = 'SELECT job_id, text, created_at FROM tweet WHERE job_id = %(job_id)s'
    job['result'] = pd.read_sql_query(sql, connection, parse_dates=['created_at'], params = {'job_id': job['job_id']})
    print(job['job_id'])

1160
2335
2616
4219
4220
4221
4253
4257
4258


In [4]:
for job in jobs:
    print(len(job['result']))

151648
843051
7318111
3716723
460743
108052
2250
748811
201828


In [7]:
from naiveBayesClassifier import tokenizer
from naiveBayesClassifier.trainer import Trainer
from naiveBayesClassifier.classifier import Classifier

trainer = Trainer(tokenizer)

<h2>Sort into training and test data</h2>
<p>We want to split the data into test and training sets.  I am going to take the first 3/4 and use that as the training data and then the remaning 1/4 of the data as test data.  To do that I need to get the total count for each job_id, if the index/len(job_id) is less than 3/4 it is added as training data, otherwise it gets added as test data.</p>

In [24]:
training = []
test = []

for job in jobs:
    total = len(job['result'])
    count = 0
    for index, result in job['result'].iterrows():
        item = {
            'description': job['description'],
            'text': result['text']
        }
        if count/total < 3/4:
            training.append(item)
        else:
            test.append(item)

In [ ]:
for item in training:
    trainer.train(item['text'], item['description'])